In [ ]:
import keras, tensorflow
from keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [ ]:
batch_size=64
epochs=100
latent_dim=256
num_samples=10000
data_path='fra.txt'

In [ ]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path,'r',encoding='utf-8') as f:
  lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines)-1)]:
  input_text,  target_text, _ = line.split('\t')
  target_text='\t'+target_text+'\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [ ]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [ ]:
print('Number of samples:',len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [ ]:
input_token_index = dict(
    [(char,i) for i, char in enumerate(input_characters)]
)
target_token_index = dict(
    [(char,i) for i, char in enumerate(target_characters)]
)




encoder_input_data=np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32'
)
decoder_input_data=np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32'
)
decoder_target_data=np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32'
)


In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] =1.
  encoder_input_data[i, t+1:, input_token_index[' ']] =1.
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]] =1.
    if t > 0:
      decoder_target_data[i, t-1, target_token_index[char]] =1.
  decoder_input_data[i, t+1:, target_token_index[' ']] =1.
  decoder_target_data[i, t:, target_token_index[' ']] =1.


In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, 
          batch_size=batch_size, epochs= epochs, validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 53s 402ms/step - loss: 1.1434 - accuracy: 0.7356 - val_loss: 1.0329 - val_accuracy: 0.7144
Epoch 2/100
125/125 [==============================] - 50s 400ms/step - loss: 0.8103 - accuracy: 0.7793 - val_loss: 0.8114 - val_accuracy: 0.7762
Epoch 3/100
125/125 [==============================] - 50s 403ms/step - loss: 0.6577 - accuracy: 0.8170 - val_loss: 0.7022 - val_accuracy: 0.7975
Epoch 4/100
125/125 [==============================] - 51s 406ms/step - loss: 0.5660 - accuracy: 0.8355 - val_loss: 0.6226 - val_accuracy: 0.8212
Epoch 5/100
125/125 [==============================] - 52s 415ms/step - loss: 0.5160 - accuracy: 0.8494 - val_loss: 0.5812 - val_accuracy: 0.8313
Epoch 6/100
125/125 [==============================] - 51s 409ms/step - loss: 0.4835 - accuracy: 0.8572 - val_loss: 0.5508 - val_accuracy: 0.8385
Epoch 7/100
125/125 [==============================] - 51s 410ms/step - loss: 0.4564 - accuracy: 0.8644 - val_loss: 0.5426 -

In [ ]:
model.save('smode.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_state_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_state_inputs,
    [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items()
)
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items()
)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
    input_sentence = "You are good"
    test_sentence_tokenized = np.zeros((1,max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    for t, char in enumerate(input_sentence):
        test_sentence_tokenized[0, t, input_token_index[char]] = 1.
    print(input_sentence)
    print(decode_sequence(test_sentence_tokenized))
    print('')

You are good
Vous êtes bonne.


